# 
<h1 style="font-size:300%; color:tomato; font-family:cursive;">Nepali House Price Prediction</h1>

The goal of this project is to learn about data pre-processing, feature engineering, and model building. I will be using the dataset from [Kaggle](https://www.kaggle.com/datasets/sagyamthapa/nepali-housing-price-dataset/data) which i felt is one of the challenging dataset which truly represents real-world noise. This notebook won’t explain the steps I take, as I have prepared a separate notebook for that.









# 1. Data Exploration

In [145]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ast

In [146]:
data = pd.read_csv('2020-4-27.csv') # read data from csv file

In [147]:
data.head(2)

,Title,Address,City,Price,Bedroom,Bathroom,Floors,Parking,Face,Year,Views,Area,Road,Road Width,Road Type,Build Area,Posted,Amenities
0,Flat System House On Sale At Budhanikantha : H...,"Budhanikantha, Budhanilkantha, Kathmandu",Kathmandu,90000000,6,3,2.0,10,West,2073.0,17,1-0-0-0 Aana,20 Feet / Blacktopped,20 Feet,Blacktopped,18-0-0-0 Aana,11 hours ago,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe..."
1,21 Aana Beautiful House : House For Sale In Bu...,"Pasikot, Budhanilkantha, Kathmandu",Kathmandu,80000000,5,3,2.0,9,East,2073.0,26,0-21-0-0 Aana,20 Feet / Blacktopped,20 Feet,Blacktopped,0-14-0-0 Aana,12 hours ago,"['Parking', 'Garage', 'Fencing', 'Frontyard', ..."


In [148]:
print(f" Columns in dataset: \n{data.columns}. \n Number of columns: {len(data.columns)}")


 Columns in dataset: 
Index(['Title', 'Address', 'City', 'Price', 'Bedroom', 'Bathroom', 'Floors',
       'Parking', 'Face', 'Year', 'Views', 'Area', 'Road', 'Road Width',
       'Road Type', 'Build Area', 'Posted', 'Amenities'],
      dtype='object'). 
 Number of columns: 18


In [149]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2211 entries, 0 to 2210
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Title       2211 non-null   object 
 1   Address     2211 non-null   object 
 2   City        2211 non-null   object 
 3   Price       2211 non-null   int64  
 4   Bedroom     2211 non-null   int64  
 5   Bathroom    2211 non-null   int64  
 6   Floors      1039 non-null   float64
 7   Parking     2211 non-null   int64  
 8   Face        2211 non-null   object 
 9   Year        582 non-null    float64
 10  Views       2211 non-null   object 
 11  Area        2211 non-null   object 
 12  Road        2211 non-null   object 
 13  Road Width  2211 non-null   object 
 14  Road Type   1426 non-null   object 
 15  Build Area  2211 non-null   object 
 16  Posted      2211 non-null   object 
 17  Amenities   2211 non-null   object 
dtypes: float64(2), int64(4), object(12)
memory usage: 311.0+ KB


In [150]:
data.shape # shape of the data

(2211, 18)

In [151]:
data.index # index of the data

RangeIndex(start=0, stop=2211, step=1)

In [152]:
data.isna().sum() # check for missing values

Title            0
Address          0
City             0
Price            0
Bedroom          0
Bathroom         0
Floors        1172
Parking          0
Face             0
Year          1629
Views            0
Area             0
Road             0
Road Width       0
Road Type      785
Build Area       0
Posted           0
Amenities        0
dtype: int64

In [153]:
data['Year'].corr(data['Price']) #As there was more missing values is "year " so checking how important is this column as our goal is yto predict the price

np.float64(0.035410409777585986)

In [154]:
data.sort_values("Year", ascending=True).head(3)

,Title,Address,City,Price,Bedroom,Bathroom,Floors,Parking,Face,Year,Views,Area,Road,Road Width,Road Type,Build Area,Posted,Amenities
1607,"Prachanda Chitrakar : Hostel For Rent In Ason,...","Thahity, Ason, Kathmandu",Kathmandu,190000150000,0,0,7.0,3,West,1994.0,352,2 Aana,20 Feet / Blacktopped,20 Feet,Blacktopped,5500 Sq. Feet,11 months ago,"['Drainage', 'Water Supply']"
1538,"DT : Office Space For Rent In Jawalakhel, Lali...","Jawalakhel, Lalitpur, Jawalakhel, Lalitpur",Lalitpur,155000150000,0,0,2.0,0,South West,2000.0,301,1-12-0-0 Aana,32 Feet / Blacktopped,32 Feet,Blacktopped,5 Aana,10 months ago,[]
243,House On Sale At Chabahil (50m Towards Ganeshs...,"Chabahil, Charumati bihar, Chabahil, Kathmandu",Kathmandu,25000000,6,4,4.0,2,East,2000.0,479,2.5 Aana,20 Feet / Paved,20 Feet,Paved,2.5 Aana,2 months ago,"['Parking', 'Balcony', 'Drainage', 'Water Supp..."


In [155]:
data[data['Year'].isna() | data['Floors'].isna()][['Year','Floors']] # checking missing values

,Year,Floors
17,NaN,NaN
18,NaN,NaN
19,NaN,NaN
21,NaN,NaN
25,NaN,3.0
...,...,...
2205,NaN,NaN
2206,NaN,3.0
2207,NaN,2.0
2209,NaN,2.0


In [156]:
data['Year'].mean()

np.float64(2057.780068728522)

# 2. Data Cleaning and Transformations

In [157]:
year_constant = data['Year'].mean().round(0)


In [158]:
floor_constant = data['Floors'].mean().round(0) 

## 2.1. Handling Missing Values

In [159]:
#filling missing value of year wiht mean value

data['Year'] = data['Year'].fillna(year_constant)

In [160]:
# filling missing vlue of floor with mean value
data['Floors'] = data['Floors'].fillna(floor_constant)

In [161]:
# handling road type column by dropping it
data.drop('Road Type', axis=1, inplace=True) # droping column

In [162]:
data.isna().sum()

Title         0
Address       0
City          0
Price         0
Bedroom       0
Bathroom      0
Floors        0
Parking       0
Face          0
Year          0
Views         0
Area          0
Road          0
Road Width    0
Build Area    0
Posted        0
Amenities     0
dtype: int64

## 2.2 Dropping Unnecessary Columns

In [163]:
data.head(2)

,Title,Address,City,Price,Bedroom,Bathroom,Floors,Parking,Face,Year,Views,Area,Road,Road Width,Build Area,Posted,Amenities
0,Flat System House On Sale At Budhanikantha : H...,"Budhanikantha, Budhanilkantha, Kathmandu",Kathmandu,90000000,6,3,2.0,10,West,2073.0,17,1-0-0-0 Aana,20 Feet / Blacktopped,20 Feet,18-0-0-0 Aana,11 hours ago,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe..."
1,21 Aana Beautiful House : House For Sale In Bu...,"Pasikot, Budhanilkantha, Kathmandu",Kathmandu,80000000,5,3,2.0,9,East,2073.0,26,0-21-0-0 Aana,20 Feet / Blacktopped,20 Feet,0-14-0-0 Aana,12 hours ago,"['Parking', 'Garage', 'Fencing', 'Frontyard', ..."


In [164]:
temp = data.pop('Title') # poping out the title column which might be needed for resolving otyher missing values

In [165]:
columns_to_drop = ['Address', 'Views', 'Posted','Road']

In [166]:
data.drop(columns_to_drop, axis=1, inplace=True)

In [167]:
data.head()

,City,Price,Bedroom,Bathroom,Floors,Parking,Face,Year,Area,Road Width,Build Area,Amenities
0,Kathmandu,90000000,6,3,2.0,10,West,2073.0,1-0-0-0 Aana,20 Feet,18-0-0-0 Aana,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe..."
1,Kathmandu,80000000,5,3,2.0,9,East,2073.0,0-21-0-0 Aana,20 Feet,0-14-0-0 Aana,"['Parking', 'Garage', 'Fencing', 'Frontyard', ..."
2,Kathmandu,70000000,5,3,2.0,12,East,2071.0,1-1-0-0 Aana,20 Feet,10-1-0-0 Aana,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe..."
3,Kathmandu,130000000,6,4,3.0,9,West,2017.0,0-10-1-0 Aana,20 Feet,0-7-1-0 Aana,"['Parking', 'Lawn', 'Garage', 'Frontyard', 'Fe..."
4,Kathmandu,100000000,6,3,2.0,10,East,2070.0,0-19-2-0 Aana,20 Feet,0-14-2-0 Aana,"['Parking', 'Lawn', 'Garage', 'Balcony', 'Back..."


In [168]:
temp = set()

for amenities in data['Amenities']:
    # Remove the brackets and split by comma
    amenities_list = amenities.strip('[]').replace("'", "").split(',')
    # Strip whitespace and add each cleaned item to the set
    temp.update(item.strip() for item in amenities_list)

temp = list(temp)
temp = temp[1:]

## 2.3 Handling Ameneities columnn

In [169]:
data['Amenities'] = data['Amenities'].apply(ast.literal_eval)

# Create dummy variables for each unique amenity
amenities_dummies = pd.get_dummies(data['Amenities'].apply(pd.Series).stack()).groupby(level=0).sum()

# Concatenate the original dataframe with the dummy variables
data = pd.concat([data, amenities_dummies], axis=1)

data = data.drop('Amenities', axis=1)

## 2.4 Convert Road Width column to Meter

In [170]:
def convert_to_meters(value):
    value = value.strip().lower()
    if 'feet' in value:
        # Extract numeric part and convert to meters
        feet = float(value.replace('feet', '').strip())
        return round(feet * 0.3048, 2)  # Convert to meters and round to 2 decimal places
    elif 'meter' in value:
        # Extract numeric part (already in meters)
        meters = float(value.replace('meter', '').strip())
        return round(meters, 2)
    else:
        # Handle cases without units (assuming it's in feet and convert to meters)
        feet = float(value)
        return round(feet * 0.3048, 2)

# Apply the function to the 'Road Width' column
data['Road Width'] = data['Road Width'].apply(convert_to_meters)

## 2.5 Handling Face column

In [175]:
face_dummies= pd.get_dummies(data['Face']).astype(int)
data = pd.concat([data, face_dummies], axis=1)
data = data.drop('Face', axis=1)



In [176]:
data

,City,Price,Bedroom,Bathroom,Floors,Parking,Year,Area,Road Width,Build Area,...,Water Well,Wifi,East,North,North East,North West,South,South East,South West,West
0,Kathmandu,90000000,6,3,2.0,10,2073.0,1-0-0-0 Aana,6.10,18-0-0-0 Aana,...,1.0,1.0,0,0,0,0,0,0,0,1
1,Kathmandu,80000000,5,3,2.0,9,2073.0,0-21-0-0 Aana,6.10,0-14-0-0 Aana,...,1.0,1.0,1,0,0,0,0,0,0,0
2,Kathmandu,70000000,5,3,2.0,12,2071.0,1-1-0-0 Aana,6.10,10-1-0-0 Aana,...,1.0,1.0,1,0,0,0,0,0,0,0
3,Kathmandu,130000000,6,4,3.0,9,2017.0,0-10-1-0 Aana,6.10,0-7-1-0 Aana,...,1.0,1.0,0,0,0,0,0,0,0,1
4,Kathmandu,100000000,6,3,2.0,10,2070.0,0-19-2-0 Aana,6.10,0-14-2-0 Aana,...,1.0,1.0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2206,Kathmandu,500000,5,5,3.0,5,2058.0,1 Ropani,4.88,N/A Sq. Feet,...,0.0,0.0,0,0,1,0,0,0,0,0
2207,Kathmandu,12000000,4,2,2.0,1,2058.0,2.5 Aana,3.96,N/A Sq. Feet,...,0.0,0.0,0,0,1,0,0,0,0,0
2208,Kathmandu,27000000,5,3,2.0,1,2071.0,8 Aana,3.66,N/A Sq. Feet,...,0.0,0.0,1,0,0,0,0,0,0,0
2209,Kathmandu,300000,9,3,2.0,4,2058.0,51 Aana,3.96,N/A Sq. Feet,...,0.0,0.0,0,1,0,0,0,0,0,0


In [177]:
data.to_csv("cleaned_dataV1.csv", index=False) 